In [51]:
import numpy as np

In [52]:
import onnx

In [53]:
import os

In [54]:
import glob

In [55]:
from onnx import numpy_helper

In [56]:
import tvm

In [57]:
from tvm import te

In [58]:
import tvm.relay as relay

In [59]:
mnist_model = onnx.load('../mnist/mnist-8.onnx')

In [60]:
# load module param and mod

In [61]:
input_name = "Input3"

In [62]:
shape_dict = {input_name: (1, 1, 28, 28)}
print(shape_dict)

{'Input3': (1, 1, 28, 28)}


In [75]:
mod, params = relay.frontend.from_onnx(mnist_model, shape_dict)

In [76]:
# func = mod["main"]

In [77]:
# func = relay.Function(func.params, relay.nn.softmax(func.body), None, func.type_params, func.attrs)

In [78]:
target = tvm.target.arm_cpu("rasp3b")
# target = tvm.target.create('llvm')
# target = tvm.target.Target('llvm')

In [79]:
with tvm.transform.PassContext(opt_level=3):
# with relay.build_config(opt_level=2):    
    mod_lib = relay.build(mod, target, params=params, mod_name='mnist')

Cannot find config for target=llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -model=bcm2837 -mtriple=armv7l-linux-gnueabihf, workload=('conv2d_nchw_spatial_pack.arm_cpu', ('TENSOR', (1, 1, 32, 32), 'float32'), ('TENSOR', (8, 1, 5, 5), 'float32'), (1, 1), (0, 0, 0, 0), (1, 1), 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -model=bcm2837 -mtriple=armv7l-linux-gnueabihf, workload=('conv2d_nchw_spatial_pack.arm_cpu', ('TENSOR', (1, 8, 18, 18), 'float32'), ('TENSOR', (16, 8, 5, 5), 'float32'), (1, 1), (0, 0, 0, 0), (1, 1), 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -keys=arm_cpu,cpu -device=arm_cpu -link-params=0 -mattr=+neon -model=bcm2837 -mtriple=armv7l-linux-gnueabihf, workload=('dense_pack.x86', ('TENSOR', (1, 256), 'float32'), ('TENSOR', (10,

In [68]:
print(mod_lib)

In [30]:
# lib export
lib_path = "../lib/mnist.so"

In [31]:
mod_lib.export_library(lib_path)
# , cc="/usr/bin/arm-linux-gnueabihf-g++"

In [32]:
# params export

In [33]:
param_path = "../lib/mnist.params"

In [34]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

In [35]:
# test module

In [36]:
test_data_dir = '../mnist/test_data_set_2'

In [37]:
inputs = []

In [38]:
inputs_num = len(glob.glob(os.path.join(test_data_dir, 'input_*.pb')))

In [39]:
for i in range(inputs_num):
    input_file = os.path.join(test_data_dir, 'input_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    inputs.append(numpy_helper.to_array(tensor))

In [40]:
loaded_lib = tvm.runtime.load_module(lib_path)

In [41]:
#loaded_param = tvm.runtime.load_param_dict(open(param_path, "rb").read())
loaded_param = bytearray(open(param_path, "rb").read())

In [42]:
# context

In [43]:
ctx = tvm.cpu()

In [44]:
module = tvm.contrib.graph_executor.GraphModule(loaded_lib["mnist"](ctx))

In [45]:
#module = tvm.contrib.graph_executor.create(loaded_json, loaded_lib, ctx)

In [46]:
module.load_params(loaded_param)

In [47]:
module.set_input("Input3", inputs[0])

# tes

In [48]:
module.run()

In [49]:
out_deploy = module.get_output(0).asnumpy()

In [50]:
print(out_deploy)

[[-2334.0884   -1113.6259    1076.5802    -860.23926   1588.5348
  -1534.3464   -2686.0781     -56.480286    74.574875  3715.3796  ]]
